In [1]:
import camelot
import glob
import numpy as np
import pandas as pd
import re

In [2]:
def simplify_store_type(store_type):
    simplified_store_type = re.match(pattern='(.*?Stores)', string=store_type).group(0)
    return simplified_store_type
    
def clean_sales_data(sales_data):
    cleaned_sales_data = sales_data.copy()
    
    # remove newlines and any accompanying spaces
    cleaned_sales_data.replace('\s?\\n\s?', ' ', regex=True, inplace=True)
    
    # replace "Directly-run Stores", "Directly run Stores", and "Own Stores" from the
    # 2015, 2017 and other files with "Directly-Run Stores"
    cleaned_sales_data.replace(
        '((?i)([a-z\s]+)(?<!Same\s))Stores', 
        'Directly-Run Stores', 
        regex=True, 
        inplace=True
    )
    
    cleaned_sales_data.replace(
        'Average Purchase', 
        'Average Purchase per Customer', 
        regex=True, 
        inplace=True
    )
    
    cleaned_sales_data.loc[1, 0] = 'Store Type' # Same or Directly-Run
    cleaned_sales_data.loc[1, 1] = 'Metric'     # Net Sales, 
    
    # forward fill Store Type column
    cleaned_sales_data.loc[2:, 0].replace('', np.nan, inplace=True)
    cleaned_sales_data = cleaned_sales_data.ffill()
    
    # extract whether Metric includes Online Sales
    cleaned_sales_data[17] = ''
    cleaned_sales_data.loc[1, 17] = 'Includes Online Sales'
    cleaned_sales_data.loc[2:, 17] = cleaned_sales_data.loc[2:, 0].apply(
        lambda s: s.__contains__('Online Sales')
    ).astype(int)
    
    # flip column if excluding Online Sales (2017 file)
    online_sales_are_flipped = cleaned_sales_data.loc[2:, 0].apply(
        lambda s: s.__contains__('excluding Online Sales')
    ).any()
    if online_sales_are_flipped:
        cleaned_sales_data[17] = cleaned_sales_data[17].replace({0: 1, 1: 0})
        
    # simplify the Store Type column to contain Same Stores and Directly-Run Stores
    cleaned_sales_data.loc[2:, 0] = cleaned_sales_data.loc[2:, 0].apply(simplify_store_type)
    
    # re-order columns
    cleaned_sales_data = cleaned_sales_data[[0, 1, 17] + list(range(2, 17))]
    return cleaned_sales_data

In [3]:
raw_sales_data_files = glob.glob('./data/raw/sales_data/*')

In [4]:
for file in raw_sales_data_files:
    raw_sales_data = camelot.read_pdf(file, split_text=True)
    cleaned_sales_data = clean_sales_data(raw_sales_data[0].df)
    
    august_year = int(re.search(pattern='(\d{4})', string=file).group(0))
    cleaned_sales_data.to_csv(
        f'./data/preprocessed/sales_data/monthly_sales_sept{august_year-1}_to_aug_{august_year}.csv',
        header=False,
        index=False
    )